# Import Libraries


In [1]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd {HOME}/drive/MyDrive/Object_Finding

/content/drive/MyDrive/Object_Finding


In [4]:
%pwd

'/content/drive/MyDrive/Object_Finding'

In [5]:
# Pip install method (recommended)

!pip install ultralytics
# !pip uninstall ultralytics

import torch
import numpy as np
import cv2
from time import time
from statistics import mode

from IPython import display as display_v1
display_v1.clear_output()

import ultralytics
ultralytics.checks()


from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from IPython.display import display, Image
import pandas as pd
import matplotlib.pyplot as plt
import pathlib

Ultralytics YOLOv8.0.190 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


# Task 1


In [ ]:
%pwd

'/content/drive/MyDrive/Object_Finding'

In [15]:
def detect_color(mask, frame):
    colors = []
    mask = cv2.resize(mask, frame.T.shape[1:])
    mask = np.dstack((mask, mask, mask))
    mask = mask*frame

    color = cv2.cvtColor(mask, cv2.COLOR_BGR2HSV)

    ar = color[:,:,0]
    ar = ar.flatten()
    # print('Loop:')
    # print('np.unique(ar) first: 'np.unique(ar))
    ar = ar[ar != 0]
    # print(len(ar))
    if len(ar) == 0:
        ar_2 = color[:,:,2]
        ar_2 = ar.flatten()
        # print('np.unique(ar) second: ', np.unique(ar_2))
        hist, bins = np.histogram(ar_2, bins = np.unique(ar_2))
        dominant_index = np.argmax(hist)
        dominant_color = bins[dominant_index]
        if dominant_color != 0:
            return 'white'
        else:
            return 'black'
    else:
        # print('np.unique(ar) third: ', np.unique(ar))
        hist, bins = np.histogram(ar, bins = np.unique(ar))
        # print(f'hist: {hist}')
        # print(f'bins: {bins}')
        dominant_index = np.argmax(hist)

        dominant_color = bins[dominant_index]

        if dominant_color < 5:
            dominant_color = 'red'
        elif dominant_color < 22:
            dominant_color = 'orange'
        elif dominant_color < 33:
            dominant_color = 'yellow'
        elif dominant_color < 78:
            dominant_color = 'green'
        elif dominant_color < 131:
            dominant_color = 'blue'
        elif dominant_color < 170:
            dominant_color = 'violet'
        else:
            dominant_color = 'red'

    return dominant_color

In [25]:
def plot_bboxes(results, frame, COLOR = '', CONFIDENCE = 0.8):
    xyxys = []
    confidences = []
    masks = []
    names = []
    color_table = {
      "red": (255, 0, 0),
      "orange": (255, 128, 0),
      "yellow": (255, 255, 0),
      "green": (0, 255, 0),
      "blue": (0, 0, 255),
      "violet": (127, 0, 255),
      "white": (255, 255, 255),
      "black": (0, 0, 0)
    }
    exist = False
    # Extract detections for person class
    for result in results:
        for i in range(len(result.boxes.data)):
            class_id = result.boxes.cls.cpu().numpy()[i]
            name = results[0].names[class_id]
            confidence = result.boxes.conf.cpu().numpy()[i]

            xyxys.append(result.boxes.xyxy.cpu().numpy().astype(int)[i])
            confidences.append(confidence)
            masks.append(results[0].masks.data[i].cpu().numpy().astype('uint8').squeeze())
            names.append(name)
    if len(xyxys) != 0:
        for i in range(len(xyxys)):
            x1, y1, x2, y2 = xyxys[i][0], xyxys[i][1], xyxys[i][2], xyxys[i][3]

            # mask = results[0].masks.data[i].cpu().numpy().astype('uint8').squeeze()
            mask = masks[i]

            if len(COLOR) == 0:
                exist = True
                frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                text = f'{names[i]},{confidences[i]: .2f}'
                cv2.putText(frame, text, (x1 + 5, y1 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            else:
                color_name = detect_color(mask, frame)
                color_value = color_table[color_name][::-1]
                if color_name == COLOR:
                  exist = True
                  frame = cv2.rectangle(frame, (x1, y1), (x2, y2), color_value, 3)
                  text = f'{color_name} {names[i]},{confidences[i]: .2f}'
                  cv2.putText(frame, text, (x1 + 5, y1 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_value, 2)
    else:
        return frame, exist
    return frame, exist

## Level 1

In [19]:
def read_video(folder_path, video_path, model, class_id = 7, COLOR = '', OBJECT_NAME = '', confidence = 0.8):
    if not os.path.exists(folder_path + "/Results"):
        os.makedirs(folder_path + "/Results")

    path = folder_path + "/Results"
    if len(OBJECT_NAME) != 0:
        if COLOR == 'no':
                path += f"/{OBJECT_NAME}"
                if not os.path.exists(path):
                    os.makedirs(path)
        else:
            path += f"/{OBJECT_NAME} "

    base_name = pathlib.Path(video_path).stem
    path += f"/{base_name}"

    if not os.path.exists(path):
        os.makedirs(path)


    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for index in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        results = model.predict(frame, classes = class_id, conf = confidence)
        frame_new = frame.copy()
        frame_new, exist = plot_bboxes(results, frame_new, COLOR = COLOR, CONFIDENCE = confidence)
        display_v1.clear_output()
        if exist:
            cv2.imwrite(f'{path}/Frame {index}.jpg', frame_new)

    cv2.destroyAllWindows()
    cap.release()

    return

In [20]:
folder_path = "/content/drive/MyDrive/Object_Finding"
model = YOLO("yolov8l-seg.pt")
if torch.cuda.is_available():
    model.to('cuda')

In [22]:
video_path = '/content/drive/MyDrive/Object_Finding/Task 1/Videos/Video 1.mp4'

read_video(folder_path, video_path, model, OBJECT_NAME = 'A truck', class_id = 7)

In [23]:
video_path = '/content/drive/MyDrive/Object_Finding/Task 1/Videos/Video 2.mp4'

read_video(folder_path, video_path, model, OBJECT_NAME = 'A truck')

# Level 2


In [26]:
video_path = '/content/drive/MyDrive/Object_Finding/Task 1/Videos/Video 1.mp4'

read_video(folder_path, video_path, model, OBJECT_NAME = 'The red truck', COLOR = 'red')

In [27]:
video_path = '/content/drive/MyDrive/Object_Finding/Task 1/Videos/Video 2.mp4'

read_video(folder_path, video_path, model, OBJECT_NAME = 'The red truck', COLOR = 'red')

# Level 3

In [28]:
model = YOLO("yolov8m_person.pt")
if torch.cuda.is_available():
    model.to('cuda')

In [41]:
video_path = '/content/drive/MyDrive/Object_Finding/Task 1/Videos/Video 3.mp4'

#read_video(folder_path, video_path, model, OBJECT_NAME = 'person')
if not os.path.exists(folder_path + "/Results"):
        os.makedirs(folder_path + "/Results")

path = folder_path + "/Results"

path += f"/Person"

base_name = pathlib.Path(video_path).stem
path += f"/{base_name}"

if not os.path.exists(path):
    os.makedirs(path)


cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

for index in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break
    result = model.predict(frame)[0]
    if len(result) != 0:
        frame_new = result.plot()
        cv2.imwrite(f'{path}/Frame {index}.jpg', frame_new)
    display_v1.clear_output()

cv2.destroyAllWindows()
cap.release()